In [97]:
import numpy as np

In [98]:

data = np.load("./HW02/data/data.npz")
Xtr = data["Xtr"] #Train inputs
Ytr = data["Ytr"] #Train labels
Xte = data["Xte"] #Test inputs
Yte = data["Yte"] #Test labels

In [99]:
def krbf(x1,x2,beta=0.1):
    return np.exp(-beta*np.linalg.norm(x1-x2)**2)

In [100]:
theta=[1,1,1]
x1=Xtr[0]
x2=Xtr[1]
print(f'X1:{x1}, X2:{x2}')

X1:[-1.04891018 -1.36930639  0.8309701   0.39146673 -0.28062196 -0.39550548
  0.9952759   0.12794098  1.39897922 -0.72993351  0.64032121 -0.6940647
 -0.90592121], X2:[-1.04891018 -1.36930639 -1.3326879  -0.07224493 -0.31790614 -0.39550548
  0.9952759   1.18362036 -0.7148069  -0.39858211  0.64032121 -0.6940647
 -0.90592121]


In [101]:
print(f'rbf(x1,x1){krbf(x1,x1)}, rbf(x2,x2):{krbf(x2,x2)}, rbf(x1,x2):{krbf(x2,x1)}')

rbf(x1,x1)1.0, rbf(x2,x2):1.0, rbf(x1,x2):0.34679623141839294


In [102]:
def gtheta(params,x,data):
    return params[-1]+np.sum([params[i]*krbf(data[i],x) for i in range(len(data))])

In [106]:
print(np.sign(gtheta(theta,x1,Xdata)))

1.0


In [300]:
def risk(params,xdata,ydata,lambdas=0.1):
    n=len(xdata)
    r1=np.sum(np.log(1+np.exp(-ydata[i]*gtheta(params,xdata[i],xdata))) for i in range(n))
    r2=np.sum([params[i]*params[j]*krbf(xdata[i],xdata[j]) for i in range(n) for j in range(n)])*lambdas/2
    return (r1+r2)/n

In [311]:
from joblib import Memory
location = './cachedir'
memory = Memory(location, verbose=0)

In [144]:
theta=[1,1,1]
Xdata=Xtr[:2]
Ydata=Ytr[:2]
print(f'Risk:{risk(theta,Xdata,Ydata)}')

Risk:[2.50550683]


/var/folders/zm/t5q0r2zn06j6v7256cjw7_j80000gn/T/ipykernel_53284/3988675311.py:3: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  r1=np.sum(np.log(1+np.exp(-ydata[i]*gtheta(params,xdata[i],xdata))) for i in range(n))


In [156]:
def risk_gradient(params,xdata,ydata,lambdas=0.1):
    gradient=[]
    n=len(xdata)
    for i in range(n):
        loss_term=[]
        regularized_term=[]
        for j in range(n):
            loss_term.append(1/(np.exp(ydata[i]*gtheta(params,xdata[i],xdata))+1)*-ydata[i]*krbf(xdata[i],xdata[j]))
            regularized_term.append(lambdas*params[j]*krbf(xdata[i],xdata[j]))
        gradient.append((np.sum(loss_term+regularized_term))/n)
    gradient.append((np.sum([1/(np.exp(ydata[i]*gtheta(params,xdata[i],xdata))+1)*-ydata[i]]))/n)
    return gradient

In [301]:
print(f'risk_gradients are :{risk_gradient(theta,Xdata,Ydata)}')

risk_gradients are :[array([0.68193629]), array([0.68193629]), 0.4563396189287508]


/opt/homebrew/lib/python3.10/site-packages/numpy/core/fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [312]:
import numpy as np
import scipy.special 
from scipy.optimize import minimize

class klr:

    def __init__(self,Xtr,lam=0.1,beta=0.1):
        self.lam  = lam
        self.beta = beta
        self.Xtr  = Xtr
        self.tol  = 1e-6
        self.method = "L-BFGS-B"
        self.theta_hat = []
        
    def compute_kernel(self,X,Xprime):
        #X.shape[0] is column and Xprime.shape[0] is rows
        krbf=np.array([[np.exp(-self.beta*np.linalg.norm(X[i]-Xprime[j])**2) for i in range(X.shape[0])] for j in range(Xprime.shape[0])])
        return krbf
    
    def discriminant(self,theta,X):
        K=self.compute_kernel(self.Xtr,X)
        return theta[-1]+K@np.array(theta[:-1])

    def risk(self,theta,X,Y):
        self.discriminant=memory.cache(self.discriminant)
        self.compute_kernel=memory.cache(self.compute_kernel)
        Discriminant=self.discriminant(theta,X)
        K=self.compute_kernel(X,X)

        theta=np.array(theta)
        n=len(X)
        r1=np.sum([np.log(1+np.exp(-Y[i]*Discriminant[i])) for i in range(n)])
        r2=np.sum([theta[i]*theta[j]*K[i][j] for i in range(n) for j in range(n)])*self.lam/2
        return (r1+r2)/n

    def risk_grad(self,theta,X,Y,K=0):
        gradient=[]
        n=len(X)
        Discriminant=self.discriminant(theta,X)
        K=self.compute_kernel(X,X)
        grad_b=[]
        for i in range(n):
            inner_gradient=[]
            for j in range(n):
                inner_gradient.append((1/(np.exp(Y[i]*Discriminant[i])+1)*-Y[i]*K[i][j]
                                +self.lam*theta[j]*K[i][j])/n)
            gradient.append(sum(inner_gradient))
            grad_b.append(-Y[i]/(1+np.exp(Y[i]*Discriminant[i])))
        gradient.append((np.sum(grad_b))/n)
        return gradient

    def fit(self,X,Y):
        theta0=np.zeros(len(X)+1)
        minimizing=minimize(self.risk,theta0,args=(X,Y),tol=self.tol,method=self.method,options={"disp":1},jac=self.risk_grad)
        self.theta_hat=minimizing.x
        return minimizing.x
        
        
    def predict(self,theta,X):
        return np.sign(self.discriminant(theta,X))

In [298]:
model=klr(Xdata)
theta=[1,1,1]
test=model.fit(Xtr,Y)
print(test)


NameError: name 'Y' is not defined

In [309]:
model=klr(Xtr[:3])
print(model.risk([1,1,1,1],Xtr[:3],Ytr[:3]))
print(model.risk_grad([1,1,1,1],Xtr[:3],Ytr[:3]))
from scipy import optimize
eps = np.sqrt(np.finfo(np.float).eps)
optimize.approx_fprime([1,1,1,1],model.risk,eps,Xtr[:3],Ytr[:3])

2.9651072534386853
[array([0.57788477]), array([0.66227487]), array([0.65490967]), 0.9432865518582494]


/var/folders/zm/t5q0r2zn06j6v7256cjw7_j80000gn/T/ipykernel_53284/3467251820.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps = np.sqrt(np.finfo(np.float).eps)


array([0.58051652, 0.66068947, 0.65386331, 0.94328654])

## Q7a

In [161]:
print(model.compute_kernel(Xte[:4],Xtr[:3]))

[[0.09357018488596104, 0.02918977022590466, 0.1500138965901967, 0.0999568775625893], [0.06968701109766869, 0.016615535257405983, 0.24288600093866713, 0.1224131519700177], [0.05922661197798164, 0.020415902155050146, 0.1527650204413431, 0.2825388210971133]]


## Q7b

In [199]:
model=klr(Xtr[:3])
print(model.discriminant([1,1,1,-0.5],Xtr[:4]))


[1.17423697 1.39642336 1.37706786 0.60775132]


## Q7c

In [274]:
model=klr(Xtr[:3])
print(model.predict([1,1,1,-0.5],Xte[:4]))

[-1. -1.  1.  1.]


## Q7d

In [275]:
model=klr(Xtr[:3])
print(model.risk([1,1,1,-0.5],Xtr[:3],Ytr[:3]))

1.6451804988265282


## Q7e

In [276]:
model=klr(Xtr[:3])
print(model.risk_grad([1,1,1,-0.5],Xtr[:3],Ytr[:3]))

[array([0.48213003]), array([0.56994837]), array([0.56219413]), 0.48600108386080776]


## Q7f

In [277]:
model=klr(Xtr[:10])
test=model.fit(Xtr[:10],Ytr[:10])


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           11     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.93147D-01    |proj g|=  2.00000D-01

At iterate    1    f=  5.07099D-01    |proj g|=  7.65469D-02

At iterate    2    f=  4.27204D-01    |proj g|=  6.54683D-02

At iterate    3    f=  2.74317D-01    |proj g|=  1.55827D-02

At iterate    4    f=  2.61855D-01    |proj g|=  7.21234D-03

At iterate    5    f=  2.58016D-01    |proj g|=  3.09474D-03

At iterate    6    f=  2.57314D-01    |proj g|=  1.69514D-03

At iterate    7    f=  2.57119D-01    |proj g|=  8.32129D-04

At iterate    8    f=  2.57046D-01    |proj g|=  3.35698D-04

At iterate    9    f=  2.57040D-01    |proj g|=  6.91280D-05

At iterate   10    f=  2.57039D-01    |proj g|=  1.66589D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cau

 This problem is unconstrained.


In [278]:
print(test)

[-1.09578631 -0.64489728 -0.7214171  -0.71084364 -1.13701259 -1.06009207
  2.15855564  2.40217648  1.96209868 -1.15250871 -0.97151541]


## Q7g

In [279]:
model=klr(Xtr)
fit=model.fit(Xtr,Ytr)

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          208     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.93147D-01    |proj g|=  5.41048D-02

At iterate    1    f=  5.75812D-01    |proj g|=  1.65306D-01

At iterate    2    f=  4.74978D-01    |proj g|=  5.28694D-02

At iterate    3    f=  4.53404D-01    |proj g|=  8.98336D-03

At iterate    4    f=  4.46255D-01    |proj g|=  1.14664D-02

At iterate    5    f=  4.34141D-01    |proj g|=  1.70360D-02

At iterate    6    f=  4.07928D-01    |proj g|=  1.49476D-02

At iterate    7    f=  3.76156D-01    |proj g|=  1.48575D-02

At iterate    8    f=  3.56464D-01    |proj g|=  5.47457D-03

At iterate    9    f=  3.44018D-01    |proj g|=  8.96758D-03

At iterate   10    f=  3.33236D-01    |proj g|=  1.07718D-02

At iterate   11    f=  3.22147D-01    |proj g|=  3.84685D-03

At iterate   12    f=  3.15088D-01    |proj g|=  1.34189D-02

At iterate   13    f=  3.1

In [280]:
print(fit)

[-1.54239797 -0.20525968 -0.19592715 -0.52002835 -1.30439086 -1.44777639
  0.45452064  2.15393805  1.02473118 -1.73668596  0.76718545 -1.17677659
 -0.87276666  0.5635416  -2.46024775 -1.36362478 -0.75055244 -0.01444547
  0.46559065  1.14850052  1.03118063 -0.31886599 -2.24994095  0.12542753
 -1.96140085 -2.15802725 -0.79076773  1.10505163  1.02567233  0.67120184
 -1.5444797  -0.85869961 -2.21959355 -0.6050796  -0.24473908  1.25320055
  0.26705266  0.59266045  5.25085384  1.40532092  1.11987444 -1.41331319
 -1.57975339  3.53905125  2.2978425  -0.3386239  -0.91341199 -3.02335667
 -0.35803641  1.57050523 -1.60351126 -1.86611192 -0.15916961  0.97921449
  5.18631853  0.43945665 -0.71278067 -1.26100883  2.98479412  1.20546951
  2.39490987 -3.27771839  0.25295996  1.38751639 -5.06803907  0.68671816
  4.1784401  -1.0435636   1.70746505 -1.10591076  0.33581963  0.78943123
 -0.75431351 -1.87017073 -2.19799595  2.07451286  0.85582593 -1.37192537
 -0.41381353  0.14871154  1.79343657  1.31681362 -1